Проект нашей команды на второй тур отборочного этапа Сириус.ИИ.

Мы разработали сервис для оценки цены недвижимости. 
Для обучения модели нам необходимы качественные и обработанные данные.

Качество данных зависит не от нас (Верим Яндексу и ВШЭ ФКН), поэтому займёся предобработкой.

## Исследование данных.

В рамках дополнительного задания нам было предложено провести исследование данных с информацией о сделках с недвижимость в Москве за определённый период времени.

Анализ будем производить с помощью средств библиотек:
- pandas - управление данными
- seaborn - анализ графиков

In [1]:
# подключим необходимые библиотеки
import pandas as pd
import seaborn as sns

### Шаги исследования.
1) Обработка отсутствующих значени.
    
    Посмотрите на отсутствующие значения и подума те, стоит ли исключать строки с ними из датасета. Если нет, то нужно придумать решение, которое позволит обрабатывать отсутствующие значения в каждом из поле.
2) Обработка лишних значений. 

    Выясните, какие значения можно убрать из датасета без потери информации. Ориентируйтесь на корреляцию между полями.

3) Выявление аномали.

    Посмотрите на распределения значени , есть ли там выбросы и аномалии? Если да, то с чем они связаны и как их обработать?

4) Сбалансированность.

    Проверьте, сбалансированы ли данные. Подума те, по каким признакам смотреть баланс.

5) Базовы отбор признаков.

    Выясните, какие признаки влияют на целевую переменную – цену, а какие нет. Это можно сделать с помощью визуализаци . Можно изучать как отдельные признаки, так и их комбинации.

6) Статистики.

    Посчита те основные статистики на основе данных, имеет смысл сделать это по годам. Сравните с современными данными по Москве (обязательно укажите, откуда брали современные данные).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns


df = pd.read_csv('train.csv', skipinitialspace=True, sep=',', decimal=',')

# Функции для перевода столбцов в целочисленный тип
def object_to_float(columns):
    for col in columns:
        df[col] = df[col].astype(float)


def float_to_int(columns):
    for col in columns:
        df[col] = df[col].astype(int)


print(df.shape)

#Заполняем пропуски медианным значением
for i in df:
    if df[i].isnull().sum() != 0:
        df[i].fillna(df[i].median(), inplace=True)
        object_to_float([i])
        float_to_int([i])


# Замена категорий в колонке на числа для проведения исследований
def replace_category_in_column(column, category, num):
    df[column] = df[column].replace(category, num)


categorial_columns = [
    'sub_area', 'product_type', 'culture_objects_top_25',
    'thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion',
    'radiation_raion', 'railroad_terminal_raion', 'big_market_raion',
    'nuclear_reactor_raion', 'detention_facility_raion', 'water_1line',
    'big_road1_1line', 'railroad_1line', 'ecology'
]


for column in categorial_columns:
    for index, category in enumerate(df[column].unique()):
        replace_category_in_column(column, category, index)

df_investment = df[df['product_type'] == 'Investment']
df_owner_occupier = df[df['product_type'] == 'OwnerOccupier']

### Обработка отсутствующих значений.

Заметим, что используемая нами модель не предназначена для обучения на данных с отсутствующими значениями. 

Поэтому нам придётся их обработать. Есть три пути решения данной проблемы:
1) Удаление пустых значений (записей, колонок) (drop)

    Плюсы:
    - Простое решение.
    
    Минусы:
    - Потеря данных. Количество данных, на которой будет обучатся модель, может сильно уменьшится. Поэтому будем выбирать между удалением строк(теряем данные) и столбцов(теряем характеристику).
2) Заполнение некоторым другим значением (медиана, мода, ср.ариф)

    Плюсы:
    - Не теряем данные.

    Минусы:
    - Заполнение данных может сильно повлиять на распределение и "запутать" модель.
    - Задача заполнения значений может оказаться трудной.

In [75]:
train = pd.read_csv('data/train.csv')
macro = pd.read_csv('data/macro.csv')

In [71]:
pd.set_option('display.max_rows', None)
print(macro.isna().sum())
pd.reset_option('display.max_rows')

timestamp                                        0
oil_urals                                        0
gdp_quart                                       90
gdp_quart_growth                                90
cpi                                             31
ppi                                             31
gdp_deflator                                   365
balance_trade                                   31
balance_trade_growth                            90
usdrub                                           3
eurrub                                           3
brent                                            3
net_capital_export                             396
gdp_annual                                       0
gdp_annual_growth                                0
average_provision_of_build_contract              0
average_provision_of_build_contract_moscow     365
rts                                             10
micex                                           10
micex_rgbi_tr                  

In [72]:
macro['students_reg_sports_share'] = macro['students_reg_sports_share'].ffill().bfill()
macro['population_reg_sports_share'] = macro['population_reg_sports_share'].ffill().bfill()

In [73]:
macro['gdp_quart'] = macro['gdp_quart'].bfill()
macro['gdp_quart_growth'] = macro['gdp_quart_growth'].fillna(0)

In [74]:
macro['cpi'] = macro['cpi'].bfill()
macro['ppi'] = macro['ppi'].bfill()
macro['gdp_deflator'] = macro['gdp_deflator'].bfill()

In [59]:
pd.set_option('display.max_rows', None)
print(macro.dtypes)
pd.reset_option('display.max_rows')

timestamp                                      object
oil_urals                                     float64
gdp_quart                                     float64
gdp_quart_growth                              float64
cpi                                           float64
ppi                                           float64
gdp_deflator                                  float64
balance_trade                                 float64
balance_trade_growth                          float64
usdrub                                        float64
eurrub                                        float64
brent                                         float64
net_capital_export                            float64
gdp_annual                                    float64
gdp_annual_growth                             float64
average_provision_of_build_contract           float64
average_provision_of_build_contract_moscow    float64
rts                                           float64
micex                       

In [76]:
macro['child_on_acc_pre_school'] = pd.to_numeric(macro['child_on_acc_pre_school'].str.replace(',', '.'), errors='coerce')
macro['modern_education_share'] = pd.to_numeric(macro['modern_education_share'].str.replace(',', '.'), errors='coerce')
macro['old_education_build_share'] = pd.to_numeric(macro['old_education_build_share'].str.replace(',', '.'), errors='coerce')

In [82]:
correlation_matrix = macro.drop(columns=['timestamp']).corr()

# Выводим матрицу корреляций
pd.set_option('display.max_rows', None); pd.set_option('display.max_columns', None)
print(correlation_matrix.shape)
pd.reset_option('display.max_rows'); pd.reset_option('display.max_columns')


# Находим скоррелированные признаки (критерий: корреляция по модулю > 0.5)
correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.9:
            print(correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j])

print("\nСкоррелированные характеристики:")
print(correlated_features)


(99, 99)
cpi gdp_quart_growth -0.900037665343264
ppi cpi 0.9660013472352312
gdp_deflator cpi 0.9540427934111695
gdp_deflator ppi 0.9367335753281398
usdrub cpi 0.9183353967015577
eurrub cpi 0.9161641369122464
eurrub usdrub 0.9908317273522679
brent oil_urals 0.9948652707715568
gdp_annual gdp_quart 0.9219751068153972
gdp_annual cpi 0.9192252119103026
gdp_annual ppi 0.9434407062329812
gdp_annual gdp_deflator 0.9848212736603967
gdp_annual_growth gdp_deflator -0.9061803437895322
micex_cbi_tr cpi 0.9656411770002595
micex_cbi_tr ppi 0.9405713750144096
micex_cbi_tr gdp_deflator 0.9446704542796316
micex_cbi_tr gdp_annual 0.9180947347860211
deposits_value cpi 0.9834498816688476
deposits_value ppi 0.9662593997299616
deposits_value gdp_deflator 0.9809525184817472
deposits_value gdp_annual 0.9604537696916746
deposits_value micex_cbi_tr 0.9737982012799624
grp gdp_quart 0.91630823509832
grp cpi 0.9719864926357799
grp ppi 0.9453569889123628
grp gdp_deflator 0.9617961019019222
grp gdp_annual 0.974509660